In [65]:
# You may need these
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline


import matplotlib.pyplot as plt
%matplotlib inline

# load the dataset

In [129]:
dataset_name = "grp9dataset.csv"

df = pd.read_csv(dataset_name)
# print(df.head())
# scaler = StandardScaler()
drops = ["Sation", "Period", "From Date", "To Date"]
df = df.drop(drops, axis='columns')[df.SO2 != "-"]
df = df.fillna(0)
# scaler.fit(df)
# df = pd.DataFrame(scaler.transform(df), columns=df.columns)
df.head()



,Latitude,Longitude,PM2.5,PM10,NO2,NOx,CO,O3,SO2
0,28.815329,77.153010,196.86,309.67,45.31,25.11,1.28,63.48,4.43
1,28.647622,77.315809,208.43,372.22,78.68,103.99,2.21,34.15,16.18
2,28.695381,77.181665,208.21,349.79,68.91,55.66,2.09,5.18,19.42
4,28.776200,77.051074,254.93,406.77,35.94,23.56,1.94,64.71,10.27
7,28.656756,77.227234,0.00,307.87,81.61,78.99,2.57,26.28,17.52


In [130]:
def seperate_dataset(data):
  target = 'NO2'
  X = data.drop(target, axis='columns')
  y = data[target].values

  print(X.head())

  return X, y

# df = normalize(df)
X, y = seperate_dataset(df)
test_size = 0.2

# # Splitting code here
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=test_size)

    Latitude  Longitude   PM2.5    PM10     NOx    CO     O3    SO2
0  28.815329  77.153010  196.86  309.67   25.11  1.28  63.48   4.43
1  28.647622  77.315809  208.43  372.22  103.99  2.21  34.15  16.18
2  28.695381  77.181665  208.21  349.79   55.66  2.09   5.18  19.42
4  28.776200  77.051074  254.93  406.77   23.56  1.94  64.71  10.27
7  28.656756  77.227234    0.00  307.87   78.99  2.57  26.28  17.52


In [132]:
# Train
def get_model():
  model = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('model', LinearRegression())
  ])

  return model

model = get_model()

print(cross_val_score(model, xtrain, ytrain, cv=5, scoring="neg_mean_squared_error").mean())

model.fit(xtrain, ytrain)

# yhat_train = model.predict(xtrain)
yhat = model.predict(xtest)

# yhat2 = np.zeros((yhat.shape[0], df.shape[1]))

# for i in range(yhat.shape[0]):
#   yhat2[i, 4] = yhat[i]

# yhat2 = pd.DataFrame(yhat2, columns=df.columns)
# print(yhat2.head())

# yhat2 = scaler.inverse_transform(yhat2)
# yhat2 = pd.DataFrame(yhat2, columns=df.columns)
# yhat = yhat2["NO2"].values
print(yhat)

# yhat = scaler.inverse_transform(yhat.reshape(-1, 1))

# train_score = model.score(ytrain, yhat_train)
# test_score = model.score(ytest.reshape(-1, 1), yhat.reshape(-1, 1))

# print(f"Train score: {train_score}, Test score: {test_score}")
# print(test_score)


-55589.13013985973
[ 179.14876128  234.32929928  161.13722887  137.90976474  368.02850288
 2665.99754014   59.26256701]


In [32]:
def plot_model(model):
  xscatter = xtest.male
  yscatter = ytest

  # xscatter, yscatter, xline, yline1, yline2 must all be 1D arrays
  # ax = sns.scatterplot(x=xscatter,y=yscatter,alpha=0.2) #Plot data
  # sns.lineplot(x=xline,y=yline_label) # Plot line indicating label
  # sns.lineplot(x=xline,y=yline_prob) # Plot line indicating probability

# Testing

In [28]:
# # Calculate performance measures from scratch
# # TP: true postives
# # TN: true negatives
# # FP: False positives
# # FN: False negatives
# def compute_performance(yhat, y, classes):
#     # First, get tp, tn, fp, fn
#     tp = sum(np.logical_and(yhat == classes[1], y == classes[1]))
#     tn = sum(np.logical_and(yhat == classes[0], y == classes[0]))
#     fp = sum(np.logical_and(yhat == classes[1], y == classes[0]))
#     fn = sum(np.logical_and(yhat == classes[0], y == classes[1]))

#     print(f"tp: {tp} tn: {tn} fp: {fp} fn: {fn}")

#     # Accuracy
#     acc = (tp + tn) / (tp + tn + fp + fn)

#     # Precision
#     # "Of the ones I labeled +, how many are actually +?"
#     precision = tp / (tp + fp)

#     # Recall
#     # "Of all the + in the data, how many do I correctly label?"
#     recall = tp / (tp + fn)

#     # Sensitivity
#     # "Of all the + in the data, how many do I correctly label?"
#     sensitivity = recall

#     # Specificity
#     # "Of all the - in the data, how many do I correctly label?"
#     specificity = tn / (fp + tn)

#     # Print results

#     print("Accuracy:",round(acc,3),"Recall:",round(recall,3),"Precision:",round(precision,3),
#           "Sensitivity:",round(sensitivity,3),"Specificity:",round(specificity,3))